In [1]:
import pandas as pd
import tensorflow_hub as hub
import openai 
import numpy as np
import altair as alt
from openai import OpenAI
from sql_metadata import Parser
import os
from IPython.display import display, Markdown
import ast
import cx_Oracle
import re

from pathlib import Path
import sys
sys.path.append('../')
import scripts.eval
from scripts.eval import execute_query 
from scripts.eval import metric 
import json
import requests
import datetime

2024-04-26 17:03:54.285852: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-26 17:03:54.286028: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-26 17:03:54.342839: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-26 17:03:54.467919: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-26 17:03:55.266027: W tensorflow/compiler/tf2

# Read csv-files

In [3]:
prefix = Path.cwd().parent.parent
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
prefix

PosixPath('/home/skikk/Dokumente/uni/WS2324/MIT/llm-enterprise')

In [4]:
with open('../../config.json') as f:
  config = json.load(f)

In [22]:
df_q = pd.read_json((str(prefix) + '/data/50_queries.json'))

In [23]:
df_q

,question,sql
0,How many organisation units in human resources are relabeled?,SELECT COUNT(*) FROM (SELECT hr_org_unit_key FROM hr_org_unit_new INTERSECT SELECT hr_org_unit_key FROM hr_org_unit);
1,How many organisation units in human resources are new and not relabeled?,SELECT COUNT(*) FROM (SELECT hr_org_unit_key FROM hr_org_unit_new MINUS SELECT hr_org_unit_key FROM hr_org_unit);
2,COUNT the number of courses offered by each department.,"SELECT OFFER_DEPT_CODE, COUNT(*) AS course_COUNT FROM SUBJECT_ENROLLABLE GROUP BY OFFER_DEPT_CODE;"
3,"What are the past, upcoming, and current courses?","SELECT DISTINCT subject_title, academic_terms.term_code, CASE WHEN term_start_date > SYSDATE THEN 'Upcoming' WHEN term_start_date <= SYSDATE AND term_end_date > SYSDATE THEN 'Current' ELSE 'Past' END AS term_status FROM SUBJECT_SUMMARY, academic_terms WHERE SUBJECT_SUMMARY.term_code = academic_terms.term_code;"
4,Which faculty has had the most location changes?,SELECT MAX(cnt) FROM (SELECT COUNT(*) AS cnt FROM WAREUSER.FCLT_BUILDING_ADDRESS_HIST GROUP BY fclt_building_key);
5,What is the average number of courses a student can enroll in?,"SELECT AVG(cnt) FROM (SELECT COUNT(*) AS cnt FROM mit_student_directory, SUBJECT_SUMMARY WHERE DEPARTMENT_CODE = department GROUP BY full_name);"
6,Which students are employed by mit?,"SELECT DISTINCT empl.full_name FROM mit_student_directory mit, employee_directory empl WHERE empl.full_name = mit.full_name;"
7,What is the shortest course available this semester?,"SELECT DISTINCT subject_title, term_end_date - term_start_date FROM (SELECT MIN(term_end_date - term_start_date) AS min_duration FROM academic_terms), academic_terms, SUBJECT_SUMMARY WHERE SUBJECT_SUMMARY.term_code = ACADEMIC_TERMS.TERM_CODE AND term_end_date - term_start_date = min_duration;"
8,Show buildings and their address.,"SELECT fb.building_name, fba.* FROM wareuser.fac_building fb INNER JOIN wareuser.fac_building_address fba ON fb.FAC_BUILDING_KEY = fba.building_key;"
9,How many students do we have?,SELECT COUNT(*) FROM mit_student_directory;


In [7]:
sqls = []
for q in df_q['sql']:
    sqls.append([q])
df_q['sql'] = sqls

In [8]:
questions = []
tables = []
sqls=[]
for i in range(len(df_q)):
    sql = []
    table = []
    for q in df_q['sql'][i]:
        q = q.upper().replace('WAREUSER.', '')
        t = Parser(q).tables
        table.append(t)
        sql.append(q.upper())
    questions.append(df_q['question'][i])
    sqls.append(sql)
    tables.append(table)
df_q = pd.DataFrame(data={"question":questions, "sql":sqls, "tables":tables})

In [9]:
df_q['tables'] = tables


## Identify the unique tables referred by the corpus 

In [10]:
unique_tables = set()
for  l in tables: 
    for t in l:
        for t_ in t:
            unique_tables.add(t_.upper())

In [11]:
unique_tables

{'ACADEMIC_TERMS',
 'ACADEMIC_TERMS_ALL',
 'CIS_COURSE_CATALOG',
 'COURSE_CATALOG_SUBJECT_OFFERED',
 'DRUPAL_EMPLOYEE_DIRECTORY',
 'EMPLOYEE_DIRECTORY',
 'FAC_BUILDING',
 'FAC_BUILDING_ADDRESS',
 'FAC_ROOMS',
 'FCLT_BUILDING',
 'FCLT_BUILDING_ADDRESS_HIST',
 'FCLT_BUILDING_HIST',
 'FCLT_MAJOR_USE',
 'FCLT_ROOMS',
 'HR_ORG_UNIT',
 'HR_ORG_UNIT_NEW',
 'IAP_SUBJECT_CATEGORY',
 'IAP_SUBJECT_DETAIL',
 'IAP_SUBJECT_PERSON',
 'IAP_SUBJECT_SESSION',
 'LIBRARY_COURSE_INSTRUCTOR',
 'LIBRARY_MATERIAL_STATUS',
 'LIBRARY_RESERVE_CATALOG',
 'LIBRARY_RESERVE_MATRL_DETAIL',
 'LIBRARY_SUBJECT_OFFERED',
 'MIT_STUDENT_DIRECTORY',
 'SE_PERSON',
 'SPACE_DETAIL',
 'STUDENT_DEPARTMENT',
 'SUBJECT_ENROLLABLE',
 'SUBJECT_OFFERED',
 'SUBJECT_SUMMARY',
 'TIP_DETAIL',
 'TIP_MATERIAL',
 'TIP_MATERIAL_STATUS',
 'TIP_SUBJECT_OFFERED',
 'TOP_LEVEL_DOMAIN',
 'ZIP_CANADA',
 'ZIP_USA'}

In [12]:
len(unique_tables)

39

In [21]:
df_q

,question,sql,tables
0,How many organisation units in human resources are relabeled?,[SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT_NEW INTERSECT SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT);],"[[HR_ORG_UNIT_NEW, HR_ORG_UNIT]]"
1,How many organisation units in human resources are new and not relabeled?,[SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT_NEW MINUS SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT);],"[[HR_ORG_UNIT_NEW, HR_ORG_UNIT]]"
2,COUNT the number of courses offered by each department.,"[SELECT OFFER_DEPT_CODE, COUNT(*) AS COURSE_COUNT FROM SUBJECT_ENROLLABLE GROUP BY OFFER_DEPT_CODE;]",[[SUBJECT_ENROLLABLE]]
3,"What are the past, upcoming, and current courses?","[SELECT DISTINCT SUBJECT_TITLE, ACADEMIC_TERMS.TERM_CODE, CASE WHEN TERM_START_DATE > SYSDATE THEN 'UPCOMING' WHEN TERM_START_DATE <= SYSDATE AND TERM_END_DATE > SYSDATE THEN 'CURRENT' ELSE 'PAST' END AS TERM_STATUS FROM SUBJECT_SUMMARY, ACADEMIC_TERMS WHERE SUBJECT_SUMMARY.TERM_CODE = ACADEMIC_TERMS.TERM_CODE;]","[[SUBJECT_SUMMARY, ACADEMIC_TERMS]]"
4,Which faculty has had the most location changes?,[SELECT MAX(CNT) FROM (SELECT COUNT(*) AS CNT FROM FCLT_BUILDING_ADDRESS_HIST GROUP BY FCLT_BUILDING_KEY);],[[FCLT_BUILDING_ADDRESS_HIST]]
5,What is the average number of courses a student can enroll in?,"[SELECT AVG(CNT) FROM (SELECT COUNT(*) AS CNT FROM MIT_STUDENT_DIRECTORY, SUBJECT_SUMMARY WHERE DEPARTMENT_CODE = DEPARTMENT GROUP BY FULL_NAME);]","[[MIT_STUDENT_DIRECTORY, SUBJECT_SUMMARY]]"
6,Which students are employed by mit?,"[SELECT DISTINCT EMPL.FULL_NAME FROM MIT_STUDENT_DIRECTORY MIT, EMPLOYEE_DIRECTORY EMPL WHERE EMPL.FULL_NAME = MIT.FULL_NAME;]","[[MIT_STUDENT_DIRECTORY, EMPLOYEE_DIRECTORY]]"
7,What is the shortest course available this semester?,"[SELECT DISTINCT SUBJECT_TITLE, TERM_END_DATE - TERM_START_DATE FROM (SELECT MIN(TERM_END_DATE - TERM_START_DATE) AS MIN_DURATION FROM ACADEMIC_TERMS), ACADEMIC_TERMS, SUBJECT_SUMMARY WHERE SUBJECT_SUMMARY.TERM_CODE = ACADEMIC_TERMS.TERM_CODE AND TERM_END_DATE - TERM_START_DATE = MIN_DURATION;]","[[ACADEMIC_TERMS, SUBJECT_SUMMARY]]"
8,Show buildings and their address.,"[SELECT FB.BUILDING_NAME, FBA.* FROM FAC_BUILDING FB INNER JOIN FAC_BUILDING_ADDRESS FBA ON FB.FAC_BUILDING_KEY = FBA.BUILDING_KEY;]","[[FAC_BUILDING, FAC_BUILDING_ADDRESS]]"
9,How many students do we have?,[SELECT COUNT(*) FROM MIT_STUDENT_DIRECTORY;],[[MIT_STUDENT_DIRECTORY]]


In [18]:
#df_q = pd.concat([df_q, new_df], ignore_index=True)
df_q[['question', 'sql', 'tables', 'gpt_sql_nocontext','gpt_sql_global_schema']]

,question,sql,tables,gpt_sql_nocontext,gpt_sql_global_schema
0,What do I have access to in the warehouse?,[SELECT * FROM ALL_VIEWS WHERE OWNER = 'WAREUSER';],[[ALL_VIEWS]],"[This question cannot be answered with a SQL query as it requires administrative access to the MIT Data Warehouse system. The access to the warehouse is determined by the user's role and permissions, which are managed by the system administrators.]","[SELECT * FROM HR_ORG_UNIT, SELECT * FROM TIP_SUBJECT_OFFERED, SELECT * FROM FAC_ROOMS, SELECT * FROM TIP_DETAIL, SELECT * FROM CIS_COURSE_CATALOG, SELECT * FROM EMPLOYEE_DIRECTORY, SELECT * FROM FAC_BUILDING, SELECT * FROM ACADEMIC_TERMS, SELECT * FROM FAC_ORGANIZATION, SELECT * FROM SPACE_SUPERVISOR_USAGE]"
1,How much space does department of engineering have?,"[SELECT HOU.HR_DEPARTMENT_NAME, FAC_BUILDING_KEY, FB.BUILDING_NUMBER, FR.FAC_ROOM_KEY, FR.ROOM, FR.AREA FROM WAREUSER.FAC_BUILDING FB JOIN WAREUSER.FAC_ROOMS FR ON FB.FAC_BUILDING_KEY = FR.BUILDING_KEY JOIN WAREUSER.FAC_ORGANIZATION FO ON FO.ORGANIZATION_KEY = FR.ORGANIZATION_KEY JOIN HR_ORG_UNIT HOU ON HOU.DLC_KEY = FO.D_CODE WHERE HOU.HR_DEPARTMENT_NAME = 'School of Engineering';]","[[FAC_BUILDING, FAC_ROOMS, FAC_ORGANIZATION, HR_ORG_UNIT]]","[SELECT SUM(space_square_feet) FROM mit_space_data WHERE department = 'Engineering', SELECT SUM(area) FROM mit_building_data WHERE department = 'Engineering']","[SELECT SUM(AREA) FROM FAC_ROOMS WHERE DEPT_CODE IN (SELECT HR_DEPARTMENT_CODE FROM HR_ORG_UNIT WHERE HR_DEPARTMENT_NAME LIKE '%Engineering%'), SELECT SUM(ASSIGNABLE_AREA) FROM FAC_BUILDING WHERE BUILDING_NUMBER IN (SELECT BUILDING_KEY FROM FAC_ROOMS WHERE DEPT_CODE IN (SELECT HR_DEPARTMENT_CODE FROM HR_ORG_UNIT WHERE HR_DEPARTMENT_NAME LIKE '%Engineering%'))]"
2,How much space does each department have?,"[SELECT HOU.HR_DEPARTMENT_NAME , SUM(FR.AREA) TOTAL_AREA_PER_BUILDING FROM WAREUSER.FAC_BUILDING FB JOIN WAREUSER.FAC_ROOMS FR ON FB.FAC_BUILDING_KEY = FR.BUILDING_KEY JOIN WAREUSER.FAC_ORGANIZATION FO ON FO.ORGANIZATION_KEY = FR.ORGANIZATION_KEY JOIN HR_ORG_UNIT HOU ON HOU.DLC_KEY = FO.D_CODE GROUP BY HOU.HR_DEPARTMENT_NAME;]","[[FAC_BUILDING, FAC_ROOMS, FAC_ORGANIZATION, HR_ORG_UNIT]]","[SELECT department_id, SUM(space) AS total_space FROM department_space GROUP BY department_id, SELECT d.department_name, SUM(s.space) AS total_space FROM department_space s JOIN departments d ON s.department_id = d.department_id GROUP BY d.department_name]","[SELECT DEPT_CODE, SUM(AREA) AS TOTAL_AREA FROM FAC_ROOMS GROUP BY DEPT_CODE, SELECT ORGANIZATION_NAME, SUM(AREA) AS TOTAL_AREA FROM FAC_ROOMS FR JOIN FAC_ORGANIZATION FO ON FR.ORGANIZATION_KEY = FO.ORGANIZATION_KEY GROUP BY ORGANIZATION_NAME, SELECT HR_DEPARTMENT_NAME, SUM(AREA) AS TOTAL_AREA FROM FAC_ROOMS FR JOIN HR_ORG_UNIT HR ON FR.DEPT_CODE = HR.HR_DEPARTMENT_CODE GROUP BY HR_DEPARTMENT_NAME, SELECT DEPARTMENT_NAME, SUM(AREA) AS TOTAL_AREA FROM FAC_ROOMS FR JOIN EMPLOYEE_DIRECTORY ED ON FR.DEPT_CODE = ED.DEPARTMENT_NUMBER GROUP BY DEPARTMENT_NAME, SELECT DEPARTMENT_NAME, SUM(AREA) AS TOTAL_AREA FROM FAC_ROOMS FR JOIN CIS_COURSE_CATALOG CC ON FR.DEPT_CODE = CC.DEPARTMENT_CODE GROUP BY DEPARTMENT_NAME]"
3,How much space does each department have in each building?,"[SELECT HOU.HR_DEPARTMENT_NAME , FB.BUILDING_NUMBER , SUM(FR.AREA) TOTAL_AREA_PER_BUILDING FROM WAREUSER.FAC_BUILDING FB JOIN WAREUSER.FAC_ROOMS FR ON FB.FAC_BUILDING_KEY = FR.BUILDING_KEY JOIN WAREUSER.FAC_ORGANIZATION FO ON FO.ORGANIZATION_KEY = FR.ORGANIZATION_KEY JOIN HR_ORG_UNIT HOU ON HOU.DLC_KEY = FO.D_CODE GROUP BY HOU.HR_DEPARTMENT_NAME, FB.BUILDING_NUMBER ORDER BY HOU.HR_DEPARTMENT_NAME;]","[[FAC_BUILDING, FAC_ROOMS, FAC_ORGANIZATION, HR_ORG_UNIT]]","[SELECT Department, Building, SUM(Space) AS TotalSpace FROM DepartmentSpaceTable GROUP BY Department, Building, SELECT d.DepartmentName, b.BuildingName, SUM(s.Space) AS TotalSpace FROM Departments d JOIN Spaces s ON d.DepartmentID = s.DepartmentID JOIN Buildings b ON s.BuildingID = b.BuildingID 

## Go thrugh the `unique_tables` list and save the respective schemas as proper CSV files 

In [267]:
for t in unique_tables: 
    if t.upper().startswith('WAREUSER.'):
        t = t[len('WAREUSER.'):]
    schema_file = str(prefix) + '/data/views/schema/' + t + '.csv'
    if os.path.isfile(schema_file): 
        s = pd.read_csv(schema_file, header=None)
        s = s.drop(1) # delete the "header" line 
        column_names = [h.strip() for h in s[0][0].split(';')]
        s_new = pd.DataFrame(list(s[0][1:].apply(lambda row: [c.strip() for c in row.split(';')])), columns=column_names)
        new_schema_file = str(prefix) + '/data/views/schema_new/' + t + '.csv'
        s_new.to_csv(new_schema_file, index=False)

## Map tables to queries 

In [13]:
queries = []
for t in unique_tables:
    q = set()
    for i, T in enumerate(df_q['tables']): 
        for j, T_ in enumerate(T):
            if t in T_: 
                q.add((i,j)) 
    queries.append(list(q))

In [14]:
queries

[[(48, 0), (32, 0)],
 [(44, 0)],
 [(4, 0)],
 [(44, 0)],
 [(9, 0), (42, 0), (37, 0), (45, 0), (35, 0), (20, 0), (5, 0), (6, 0)],
 [(15, 0), (34, 0)],
 [(2, 0)],
 [(42, 0)],
 [(32, 0)],
 [(49, 0), (14, 0)],
 [(1, 0), (0, 0)],
 [(22, 0), (11, 0)],
 [(36, 0), (38, 0)],
 [(39, 0)],
 [(43, 0), (15, 0), (32, 0)],
 [(48, 0), (32, 0)],
 [(21, 0)],
 [(1, 0), (0, 0)],
 [(10, 0)],
 [(17, 0), (8, 0)],
 [(8, 0), (14, 0)],
 [(28, 0)],
 [(17, 0)],
 [(45, 0), (13, 0), (46, 0)],
 [(45, 0), (13, 0), (46, 0), (47, 0), (39, 0)],
 [(10, 0)],
 [(25, 0), (37, 0), (17, 0), (23, 0), (18, 0), (36, 0), (6, 0), (39, 0)],
 [(40, 0)],
 [(31, 0)],
 [(31, 0)],
 [(33, 0), (31, 0)],
 [(47, 0)],
 [(44, 0)],
 [(26, 0), (24, 0)],
 [(16, 0),
  (12, 0),
  (7, 0),
  (29, 0),
  (19, 0),
  (3, 0),
  (5, 0),
  (27, 0),
  (43, 0)],
 [(7, 0), (29, 0), (34, 0), (3, 0)],
 [(22, 0), (11, 0)],
 [(44, 0)],
 [(30, 0), (41, 0)]]

In [15]:
df_t = pd.DataFrame(columns=['table', 'log'])
df_t['table'] = list(unique_tables)
df_t['log'] = queries
df_t

,table,log
0,IAP_SUBJECT_PERSON,"[(48, 0), (32, 0)]"
1,LIBRARY_RESERVE_MATRL_DETAIL,"[(44, 0)]"
2,FCLT_BUILDING_ADDRESS_HIST,"[(4, 0)]"
3,LIBRARY_SUBJECT_OFFERED,"[(44, 0)]"
4,MIT_STUDENT_DIRECTORY,"[(9, 0), (42, 0), (37, 0), (45, 0), (35, 0), (20, 0), (5, 0), (6, 0)]"
5,COURSE_CATALOG_SUBJECT_OFFERED,"[(15, 0), (34, 0)]"
6,SUBJECT_ENROLLABLE,"[(2, 0)]"
7,DRUPAL_EMPLOYEE_DIRECTORY,"[(42, 0)]"
8,IAP_SUBJECT_SESSION,"[(32, 0)]"
9,SPACE_DETAIL,"[(49, 0), (14, 0)]"


# Treat ChatGPT as an all-knowing oracle (without any additional information)
## Prompt
_Your job is to write SQL queries that answer a user's question using the tables in the MIT Data Warehouse. 
The MIT Data Warehouse is a central data source that combines data from various administrative systems at 
MIT, containing information about students, faculty, and personnel. You can find more about the MIT data 
warehouse tables at https://web.mit.edu/warehouse/metadata/tables/all_tables.html._

_How many organisation units in human resources are relabeled?_

In [19]:
api_key = config['api-key']
client = OpenAI(api_key=api_key)

## Populate the prompt and call the Open AI API

In [18]:
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {api_key}'
}

response = requests.get('https://api.openai.com/v1/engines', headers=headers)

if response.status_code == 200:
    engines = response.json()['data']
    print("Available engines:")
    for engine in engines:
        print(engine['id'])
else:
    print("Error:", response.text)

Available engines:
gpt-3.5-turbo-16k-0613
whisper-1
davinci-002
gpt-3.5-turbo-16k
dall-e-2
tts-1-hd-1106
tts-1-hd
gpt-4
dall-e-3
gpt-4-1106-preview
gpt-4-0613
gpt-3.5-turbo-instruct-0914
gpt-3.5-turbo-instruct
tts-1
gpt-3.5-turbo-0613
gpt-3.5-turbo-0301
babbage-002
tts-1-1106
text-embedding-3-large
gpt-3.5-turbo
gpt-3.5-turbo-0125
gpt-4-1106-vision-preview
text-embedding-3-small
gpt-4-vision-preview
text-embedding-ada-002
gpt-4-turbo-2024-04-09
gpt-3.5-turbo-1106
gpt-4-turbo-preview
gpt-4-0125-preview
gpt-4-turbo


In [ ]:
# TODO: do this in batch 
gpt_sql_nocontext = []
for q in df_q['question']: 
    response = client.chat.completions.create(
        model="gpt-4",
        temperature=0,
        messages = [
          {
          "role": "system",
          "content": "Your job is to write SQL queries that answer a user's question using the tables in the MIT Data Warehouse. \
          The MIT Data Warehouse is a central data source that combines data from various administrative systems at MIT, containing \
          information about students, faculty, and personnel. You can find more about the MIT data warehouse tables at \
          https://web.mit.edu/warehouse/metadata/tables/all_tables.html. \
          Reply with as many answers \
          as you can think of in SQL and include no linebreaks, newlines, escape characters or other commentary for the each possible \
          answer, but seperate the possibilities by semicolons."
          # Provide your answer in JSON form."
          # Reply with only the answer in JSON form and include no other commentary"
          },
            {"role":"user", 
             "content":q}]
    )
    gpt_sql_nocontext.append(response.choices[0].message.content)

In [246]:
sql_response = []
for q in gpt_sql_nocontext:
    sql_response.append(q.split(';'))
for i in range(len(sql_response)):
    if '' in sql_response[i]:sql_response[i] = sql_response[i][:-1]
sql_response

[["This question cannot be answered with a SQL query as it requires administrative access to the MIT Data Warehouse system. The access to the warehouse is determined by the user's role and permissions, which are managed by the system administrators."],
 ["SELECT SUM(space_square_feet) FROM mit_space_data WHERE department = 'Engineering'",
  " SELECT SUM(area) FROM mit_building_data WHERE department = 'Engineering'"],
 ['SELECT department_id, SUM(space) AS total_space FROM department_space GROUP BY department_id',
  ' SELECT d.department_name, SUM(s.space) AS total_space FROM department_space s JOIN departments d ON s.department_id = d.department_id GROUP BY d.department_name'],
 ['SELECT Department, Building, SUM(Space) AS TotalSpace FROM DepartmentSpaceTable GROUP BY Department, Building',
  ' SELECT d.DepartmentName, b.BuildingName, SUM(s.Space) AS TotalSpace FROM Departments d JOIN Spaces s ON d.DepartmentID = s.DepartmentID JOIN Buildings b ON s.BuildingID = b.BuildingID GROUP BY d

In [247]:
len(sql_response)

11

In [248]:
df_q['gpt_sql_nocontext']=sql_response

# Give ChatGPT the "database" schema as context and elicit its answer


In [16]:
def table_to_sql_create(t): 
    schema_file=str(prefix) + '/data/views/schema_new/' + t + '.csv'
    p = None
    if os.path.isfile(schema_file):
        df_t = pd.read_csv(schema_file)
        p = "CREATE TABLE "+ t+ "(\n"  
        p = p + '\n'.join(list(df_t[['COLUMN_NAME', 'DATA_TYPE']].apply(lambda row: ' '.join(row.astype(str)), axis=1)))
        p = p + ')\n'
    return p
    

In [17]:
prompt = "** all the tables in the database ** \n\n"
cnt = 0 
for t in unique_tables:
    if t.upper().startswith('WAREUSER.'):
        t = t[len('WAREUSER.'):]
    p = table_to_sql_create(t)
    if not p: 
        print(t)
    if p: 
        cnt = cnt + 1
        prompt = prompt + p
        prompt = prompt + "\n\n"

In [20]:
print(len(prompt))
additional_space = 3000-len(prompt)

20701


## Prompt
_Your job is to write SQL queries that answer a user's question using the tables in the MIT Data Warehouse. 
The MIT Data Warehouse is a central data source that combines data from various administrative systems at MIT, containing 
information about students, faculty, and personnel. The schemas of all the tables that you might need 
for answering the user questions are below. There are 50 tables in the database.  Reply with as many answers 
as you can think of in SQL and include no linebreaks, newlines, escape characters or other commentary for the each possible 
answer, but seperate the possibilities by semicolons. + "alltable_schemas"_

In [ ]:
gpt_sql_global_schema = []
for q in df_q['question']: 
    response = client.chat.completions.create(
        model="gpt-4",
        temperature = 0,
        messages = [
          {
          "role": "system",
          "content": "Your job is to write SQL queries that answer a user's question using the tables in the MIT Data Warehouse. \
          The MIT Data Warehouse is a central data source that combines data from various administrative systems at MIT, containing \
          information about students, faculty, and personnel. The schemas of all the tables that you might need \
          for answering the user questions are below. There are 50 tables in the database.  Reply with as many answers \
           as you can think of in SQL and include no linebreaks, newlines, escape characters or other commentary for the each possible \
           answer, but seperate the possibilities by semicolons. \n\n" + prompt 
          # Provide your answer in JSON form."
          # Reply with only the answer in JSON form and include no other commentary"
        },
          {"role":"user", 
           "content":q}]
          )
    gpt_sql_global_schema.append(response.choices[0].message.content)
    print(response.choices[0].message.content)

In [ ]:
sql_response = []
for q in gpt_sql_global_schema:
    sql_response.append(q.split(';'))
for i in range(len(sql_response)):
    if '' in sql_response[i]:sql_response[i] = sql_response[i][:-1]
sql_response

In [ ]:
df_q['gpt_sql_global_schema'] = sql_response

# Give ChatGPT the "database" schema and prev queries as context and elicit its answer
### Prompt
_Your job is to write SQL queries that answer a user's question using the tables in the MIT Data Warehouse. 
The MIT Data Warehouse is a central data source that combines data from various administrative systems at MIT, containing 
information about students, faculty, and personnel. The schemas of the all the tables that you might need 
for answering the user questions are given below. There are 47 tables in the database. 
For each table, the schema of the table and the previous queries that referred to the table are also given. \
Reply with as many answers as you can think of in SQL and include no linebreaks, newlines, escape characters or other 
commentary for the each possible  answer, but seperate the possibilities by semicolons_

In [40]:
def table_to_sql_create_with_logs(t, l): 
    schema_file= str(prefix) + '/data/views/schema_new/' + t + '.csv'
    p = None
    p_ = None
    if os.path.isfile(schema_file):
        df_t_ = pd.read_csv(schema_file)
        p = "CREATE TABLE " + t + "(\n" 
        p = p + '\n'.join(list(df_t_[['COLUMN_NAME', 'DATA_TYPE']].apply(lambda row: ' '.join(row.astype(str)), axis=1)))
        p = p + ')\n'
        if l:
            p = p + "\n ** PREVIOUS SQL QUERIES THAT REFERRED TO THE TABLE {0} **\n\n".format(t)
            for (i,j) in l: 
                # p = p + "QUESTION: {0} \n".format(df_q['question'][i])
                # p = p + "SQL ANSWER: {0} \n\n".format(df_q['sql'][i])
                p = p + "SQL: {0} \n\n".format(df_q['sql'][i][j])
    return p

In [49]:
def create_prompt(curr_query_index): 
    prompt = "*** ALL THE TABLES IN THE DATABASE and QUERY LOGS *** \n"
    cnt = 0 
    for i, t in enumerate(unique_tables):
        l = set(df_t['log'][i]).difference([(curr_query_index, j) for j in range(len(df_q['sql'][curr_query_index]))])
        p = table_to_sql_create_with_logs(t, list(l))
        if not p:
            print(t)
        else: 
            cnt = cnt + 1
            prompt = prompt + p + "\n"
    return prompt 

In [52]:
gpt_sql_log = []
for i, q in enumerate(df_q['question']): 
    prompt = create_prompt(i)
    print(prompt)
    response = client.chat.completions.create(
      model="gpt-4",
      messages = [
          {
          "role": "system",
          "content": "Your job is to write SQL queries that answer a user's question using the tables in the MIT Data Warehouse. \
          The MIT Data Warehouse is a central data source that combines data from various administrative systems at MIT, containing \
          information about students, faculty, and personnel. The schemas of the all the tables that you might need \
          for answering the user questions are given below. There are 47 tables in the database. \
          For each table, the schema of the table and the previous queries that referred to the table are also given. \
          \
          Reply with as many answers as you can think of in SQL and include no linebreaks, newlines, escape characters or other \
          commentary for the each possible  answer, but seperate the possibilities by semicolons. \n\n" + prompt 
          # prompt 
          # Provide your answer in JSON form."
          # Reply with only the answer in JSON form and include no other commentary"
        },
          {"role":"user", 
           "content":q}]
          )
    print(response.choices[0].message.content)
    gpt_sql_log.append(response.choices[0].message.content)

*** ALL THE TABLES IN THE DATABASE and QUERY LOGS *** 
CREATE TABLE STAR(
STAR_ID NUMBER
STAR_NAME VARCHAR2
DESCRIPTION VARCHAR2
BUSINESS_AREA_ID NUMBER
URL_OF_MODEL VARCHAR2)

 ** PREVIOUS SQL QUERIES THAT REFERRED TO THE TABLE STAR **

SQL: SELECT DISTINCT ST.STAR_NAME, REPLACE (REPLACE (ST.DESCRIPTION, CHR(13)), CHR(10), ' ') AS STAR_DESCRIPTION,  UV.VIEW_NAME, REPLACE (REPLACE (UV.DESCRIPTION, CHR(13)), CHR(10), ' ') AS VIEW_DESCRIPTION, F.FIELD_NAME, F.DEFINITION AS FIELD_DESCRIPTION  FROM STAR ST JOIN STAR_USER_VIEW SAV ON ST.STAR_ID = SAV.STAR_ID JOIN USER_VIEW UV ON SAV.VIEW_ID = UV.VIEW_ID JOIN FIELDS F ON F.TABLE_NAME = UV.VIEW_NAME ORDER BY 1, 3, 4; 


CREATE TABLE FAC_BUILDING(
DATE_ACQUIRED VARCHAR2
DATE_OCCUPIED VARCHAR2
WAREHOUSE_LOAD_DATE DATE
NUM_OF_ROOMS NUMBER
FAC_BUILDING_KEY VARCHAR2
BUILDING_NUMBER VARCHAR2
PARENT_BUILDING_NUMBER VARCHAR2
PARENT_BUILDING_NAME VARCHAR2
PARENT_BUILDING_NAME_LONG VARCHAR2
BUILDING_NAME_LONG VARCHAR2
EXT_GROSS_AREA NUMBER
ASSIGNABLE_A

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens. However, your messages resulted in 11524 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

# READ DATAFRAME


In [5]:
df_q = pd.read_csv(str(prefix) + '/data/gpt_eval_Brandin_v0.1.csv')

In [9]:
new_row = {'question':"What kind of questions can I ask the data warehoues?", 'sql':["SELECT DISTINCT ST.STAR_NAME, REPLACE (REPLACE (ST.DESCRIPTION, CHR(13)), CHR(10), ' ') AS STAR_DESCRIPTION,  UV.VIEW_NAME, REPLACE (REPLACE (UV.DESCRIPTION, CHR(13)), CHR(10), ' ') AS VIEW_DESCRIPTION, F.FIELD_NAME, F.DEFINITION AS FIELD_DESCRIPTION  FROM STAR ST JOIN STAR_USER_VIEW SAV ON ST.STAR_ID = SAV.STAR_ID JOIN USER_VIEW UV ON SAV.VIEW_ID = UV.VIEW_ID JOIN FIELDS F ON F.TABLE_NAME = UV.VIEW_NAME ORDER BY 1, 3, 4;"],
       'tables':[], 'gpt_sql_nocontext':[],
       'gpt_sql_global_schema':[], 'expected_responses':[],
       'responses_gpt_nocontent':[], 'failures_gpt_nocontent':[],
       'accuracies_nocontext':[], 'precisions_nocontext':[], 'recalls_nocontext':[],
       'f1_scores_nocontext':[], 'responses_gpt_global_schema':[],
       'failures_gpt_global_schema':[], 'accuracies_global_schema':[],
       'precisions_global_schema':[], 'recalls_global_schema':[],
       'f1_scores_global_schema':[]}
new_df = pd.DataFrame([new_row])
df_q = pd.concat([df_q, new_df], ignore_index=True)

In [8]:

df_q['sql'][10] 

["WITH SUBJECT AS ( SELECT TIP_SUBJECT_OFFERED.SUBJECT_ID SUBJECT_ID , TIP_SUBJECT_OFFERED.IS_NO_COURSE_MATERIAL IS_NO_COURSE_MATERIAL , TIP_SUBJECT_OFFERED.OFFER_DEPT_CODE OFFER_DEPT_CODE , TIP_SUBJECT_OFFERED.OFFER_DEPT_NAME OFFER_DEPT_NAME , TIP_SUBJECT_OFFERED.OFFER_SCHOOL_NAME OFFER_SCHOOL_NAME , ACADEMIC_TERMS.TERM_CODE TERM_CODE , ACADEMIC_TERMS.TERM_SELECTOR TERM_SELECTOR , CASE WHEN TIP_DETAIL.SUBJECT_ID IS NULL THEN 0 ELSE 1 END AS SUBMITTED , TIP_SUBJECT_OFFERED.SUBJECT_TITLE SUBJECT_TITLE , TIP_SUBJECT_OFFERED.MASTER_SUBJECT_ID MASTER_SUBJECT_ID , TIP_SUBJECT_OFFERED.MASTER_COURSE_NUMBER MASTER_COURSE_NUMBER , TIP_SUBJECT_OFFERED.MASTER_COURSE_NUMBER_DESC MASTER_COURSE_NUMBER_DESC , TIP_SUBJECT_OFFERED.RESPONSIBLE_FACULTY_NAME RESPONSIBLE_FACULTY_NAME , LOWER(KRB_PERSON.EMAIL_ADDRESS) c14 , SUM(TIP_SUBJECT_OFFERED.NUM_ENROLLED_STUDENTS) NUM_ENROLLED_STUDENTS , TIP_SUBJECT_OFFERED.SUBJECT_ID_SORT SUBJECT_ID_SORT , TIP_SUBJECT_OFFERED.MASTER_SUBJECT_ID_SORT MASTER_SUBJECT_ID_

In [5]:
df_q.keys()

Index(['question', 'sql', 'tables', 'gpt_sql_nocontext',
       'gpt_sql_global_schema', 'expected_responses',
       'responses_gpt_nocontent', 'failures_gpt_nocontent',
       'accuracies_nocontext', 'precisions_nocontext', 'recalls_nocontext',
       'f1_scores_nocontext', 'responses_gpt_global_schema',
       'failures_gpt_global_schema', 'accuracies_global_schema',
       'precisions_global_schema', 'recalls_global_schema',
       'f1_scores_global_schema'],
      dtype='object')

In [17]:
for q in df_q['gpt_sql_global_schema']:
    print(q[0])

SELECT * FROM HR_ORG_UNIT
SELECT SUM(AREA) FROM FAC_ROOMS WHERE DEPT_CODE IN (SELECT HR_DEPARTMENT_CODE FROM HR_ORG_UNIT WHERE HR_DEPARTMENT_NAME LIKE '%Engineering%')
SELECT DEPT_CODE, SUM(AREA) AS TOTAL_AREA FROM FAC_ROOMS GROUP BY DEPT_CODE
SELECT DEPT_CODE, BUILDING_KEY, SUM(AREA) AS TOTAL_AREA FROM FAC_ROOMS GROUP BY DEPT_CODE, BUILDING_KEY
SELECT FAC_BUILDING.BUILDING_NUMBER, COUNT(FAC_ROOMS.ROOM), SUM(FAC_ROOMS.AREA) FROM FAC_BUILDING JOIN FAC_ROOMS ON FAC_BUILDING.BUILDING_KEY = FAC_ROOMS.BUILDING_KEY WHERE FAC_ROOMS.DEPT_CODE = 'department_code' GROUP BY FAC_BUILDING.BUILDING_NUMBER
SELECT ROOM, MAJOR_USE_DESC FROM FAC_ROOMS
SELECT ROOM, LATITUDE_WGS, LONGITUDE_WGS FROM FAC_ROOMS
SELECT BUILDING_NAME, DATE_BUILT, (YEAR(CURRENT_DATE) - YEAR(DATE_BUILT)) AS BUILDING_AGE FROM FAC_BUILDING
SELECT SUM(SQFT) FROM SPACE_SUPERVISOR_USAGE
SELECT YEAR FROM (SELECT DATE_BUILT AS YEAR, COUNT(FAC_BUILDING_KEY) AS NUM_BUILDINGS, COUNT(FAC_BUILDING_KEY) - LAG(COUNT(FAC_BUILDING_KEY), 1) OVER

In [6]:
sql = []
tables = []
gpt_sql_nocontext = []
gpt_sql_global_schema = []
expected_responses = []
for i in range(len(df_q)):
    sql.append(ast.literal_eval(df_q['sql'][i]))
    tables.append(ast.literal_eval(df_q['tables'][i]))
    gpt_sql_nocontext.append(ast.literal_eval(df_q['gpt_sql_nocontext'][i]))
    gpt_sql_global_schema.append(ast.literal_eval(df_q['gpt_sql_global_schema'][i]))
    #expected_responses.append(ast.literal_eval(df_q['expected_responses'][i]))
    #responses_gpt_nocontent.append(ast.literal_eval(df_q['responses_gpt_nocontent'][i]))
    #responses_gpt_global_schema.append(ast.literal_eval(df_q['responses_gpt_global_schema'][i]))
    #tables_referred_by_gpt_sql_global_schema.append(ast.literal_eval(df_q['tables_referred_by_gpt_sql_global_schema'][i]))
    #tables_referred_by_gpt_sql_nocontext.append(ast.literal_eval(df_q['tables_referred_by_gpt_sql_nocontext'][i]))
    #sql_outputsizes.append(ast.literal_eval(df_q['sql_outputsizes'][i]))
df_q['sql'] = sql
df_q['tables'] = tables
df_q['gpt_sql_global_schema'] = gpt_sql_global_schema
df_q['gpt_sql_nocontext'] = gpt_sql_nocontext
#df_q['sql_outputsizes'] = sql_outputsizes

In [ ]:
sql = []
tables = []
gpt_sql_global_schema = []
gpt_sql_nocontext = []
question = []
gpt_global_schema = []
for i in range(len(df_q)):
    if i == 0 or df_q['question'][i] != df_q['question'][i-1]:
        if gpt_global_schema != None and i != 0:
            gpt_sql_global_schema.append(gpt_global_schema)
        gpt_global_schema = []
        sql.append(df_q['sql'][i])
        tables.append(df_q['tables'][i])
        gpt_global_schema.append(df_q['gpt_sql_global_schema'][i])
        gpt_sql_nocontext.append(df_q['gpt_sql_nocontext'][i])
        question.append(df_q['question'][i])
    else:
        gpt_global_schema.append(df_q['gpt_sql_global_schema'][i])
gpt_sql_global_schema.append(gpt_global_schema)
df_temp = pd.DataFrame(data={"question":question, "sql":sql, "tables":tables, "gpt_sql_global_schema":gpt_sql_global_schema, "gpt_sql_nocontext":gpt_sql_nocontext})

In [ ]:
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
df_q

# Generated for all answers the actual output

## Start with the correct answers

In [7]:
gold_answers = []
failures = []
for qs in df_q['sql']:
    gold_answer = []
    failure = []
    print(qs)
    for q in qs:
        try:
            gold_answer.append(execute_query(q[:-1], [v.upper() for v in Parser(q).tables]))
            failure.append(None)
        except Exception as e:
            gold_answer.append(None)
            failure.append(str(e))
    print(failure)
    gold_answers.append(gold_answer)
    failures.append(failure)

["SELECT * FROM ALL_VIEWS WHERE OWNER = 'WAREUSER';"]
[None]
["SELECT HOU.HR_DEPARTMENT_NAME, FAC_BUILDING_KEY, FB.BUILDING_NUMBER, FR.FAC_ROOM_KEY, FR.ROOM, FR.AREA FROM WAREUSER.FAC_BUILDING FB JOIN WAREUSER.FAC_ROOMS FR ON FB.FAC_BUILDING_KEY = FR.BUILDING_KEY JOIN WAREUSER.FAC_ORGANIZATION FO ON FO.ORGANIZATION_KEY = FR.ORGANIZATION_KEY JOIN HR_ORG_UNIT HOU ON HOU.DLC_KEY = FO.D_CODE WHERE HOU.HR_DEPARTMENT_NAME = 'School of Engineering';"]
[None]
['SELECT HOU.HR_DEPARTMENT_NAME , SUM(FR.AREA) TOTAL_AREA_PER_BUILDING FROM WAREUSER.FAC_BUILDING FB JOIN WAREUSER.FAC_ROOMS FR ON FB.FAC_BUILDING_KEY = FR.BUILDING_KEY JOIN WAREUSER.FAC_ORGANIZATION FO ON FO.ORGANIZATION_KEY = FR.ORGANIZATION_KEY JOIN HR_ORG_UNIT HOU ON HOU.DLC_KEY = FO.D_CODE GROUP BY HOU.HR_DEPARTMENT_NAME;']
[None]
['SELECT HOU.HR_DEPARTMENT_NAME , FB.BUILDING_NUMBER , SUM(FR.AREA) TOTAL_AREA_PER_BUILDING FROM WAREUSER.FAC_BUILDING FB JOIN WAREUSER.FAC_ROOMS FR ON FB.FAC_BUILDING_KEY = FR.BUILDING_KEY JOIN WAREUSER.FA

In [11]:
df_q['expected_responses'] = gold_answers
if not all([None in f for f in failures]):
    print("Please check errors!")

## Continue with the answers from gpt with no additional information

In [12]:
gold_answers = []
failures = []
for qs in df_q['gpt_sql_nocontext']:
    gold_answer = []
    failure = []
    for q in qs:
        try:
            gold_answer.append(execute_query(q, [v.upper() for v in Parser(q).tables]))
            failure.append(None)
        except Exception as e:
            gold_answer.append(None)
            failure.append(str(e))
    gold_answers.append(gold_answer)
    failures.append(failure)

Not supported query type: This question cannot be answered with a SQL query as it requires administrative access to the MIT Data Warehouse system. The access to the warehouse is determined by the user's role and permissions, which are managed by the system administrators.


SELECT SUM(space_square_feet) FROM mit_space_data WHERE department = 'Engineering'
An error occurred: ORA-00942: table or view does not exist
An error occurred: ORA-00942: table or view does not exist
 SELECT SUM(area) FROM mit_building_data WHERE department = 'Engineering'
An error occurred: ORA-00942: table or view does not exist
An error occurred: ORA-00942: table or view does not exist
SELECT department_id, SUM(space) AS total_space FROM department_space GROUP BY department_id
An error occurred: ORA-00942: table or view does not exist
An error occurred: ORA-00942: table or view does not exist
 SELECT d.department_name, SUM(s.space) AS total_space FROM department_space s JOIN departments d ON s.department_id = d.department_id GROUP BY d.department_name
 SELECT d.department_name, SUM(s.space) AS total_space FROM department_space s JOIN departments d ON s.department_id = d.department_id GROUP BY d.department_name
An error occurred: ORA-00942: table or view does not exist
An error occu

Not supported query type: As an AI, I can't answer this question. However, you can ask a variety of questions related to the data stored in the MIT Data Warehouse. For example, you can ask about student demographics, course enrollments, faculty details, financial data, research statistics, etc. The specific questions you can ask depend on the tables and fields available in the data warehouse.


SELECT (COUNT(DISTINCT course_id)* 100.0 / (SELECT COUNT(*) FROM courses)) AS percentage FROM tip_data WHERE term_code = 'given_term_code'
SELECT (COUNT(DISTINCT course_id)* 100.0 / (SELECT COUNT(*) FROM courses)) AS percentage FROM tip_data WHERE term_code = 'given_term_code'
An error occurred: ORA-00942: table or view does not exist
An error occurred: ORA-00942: table or view does not exist
 SELECT (COUNT(DISTINCT t.course_id)* 100.0 / (SELECT COUNT(*) FROM courses c)) AS percentage FROM tip_data t INNER JOIN courses c ON t.course_id = c.course_id WHERE t.term_code = 'given_term_code'
 SELECT (COUNT(DISTINCT t.course_id)* 100.0 / (SELECT COUNT(*) FROM courses c)) AS percentage FROM tip_data t INNER JOIN courses c ON t.course_id = c.course_id WHERE t.term_code = 'given_term_code'
An error occurred: ORA-00942: table or view does not exist
An error occurred: ORA-00942: table or view does not exist


In [13]:
df_q['responses_gpt_nocontent'] = gold_answers
df_q['failures_gpt_nocontent'] = failures

## Continue with the answers from gpt with schema information

In [14]:
gold_answers = []
failures = []
for qs in df_q['gpt_sql_global_schema']:
    gold_answer = []
    failure = []
    for q in qs:
        try:
            gold_answer.append(execute_query(q, [v.upper() for v in Parser(q).tables]))
            failure.append(None)
        except Exception as e:
            gold_answer.append(None)
            failure.append(str(e))
    print(failure)
    gold_answers.append(gold_answer)
    failures.append(failure)

 SELECT * FROM WAREUSER.SPACE_SUPERVISOR_USAGE
[None, None, None, None, None, None, None, 'ORA-12170: TNS:Connect timeout occurred', None, None]
[None, None]
 SELECT ORGANIZATION_NAME, SUM(AREA) AS TOTAL_AREA FROM WAREUSER.FAC_ROOMS FR JOIN FAC_ORGANIZATION FO ON FR.ORGANIZATION_KEY = FO.ORGANIZATION_KEY GROUP BY ORGANIZATION_NAME
 SELECT ORGANIZATION_NAME, SUM(AREA) AS TOTAL_AREA FROM WAREUSER.FAC_ROOMS FR JOIN WAREUSER.FAC_ORGANIZATION FO ON FR.ORGANIZATION_KEY = FO.ORGANIZATION_KEY GROUP BY ORGANIZATION_NAME
An error occurred: ORA-00918: column ambiguously defined
An error occurred: ORA-00918: column ambiguously defined
[None, 'First error without replacing tables: ORA-00918: column ambiguously defined \n Second Error when replacing tables: ORA-00918: column ambiguously defined.', None, None, None]
[None]
SELECT WAREUSER.FAC_BUILDING.BUILDING_NUMBER, COUNT(FAC_ROOMS.ROOM), SUM(FAC_ROOMS.AREA) FROM WAREUSER.FAC_BUILDING JOIN FAC_ROOMS ON WAREUSER.FAC_BUILDING.BUILDING_KEY = FAC_ROOMS

Not supported query type: You can ask a variety of questions related to the data stored in the MIT Data Warehouse. Here are some examples:

1. How many courses are offered in the fall term?
2. What is the total number of rooms in a specific building?
3. Who are the instructors for a specific course?
4. What is the total area of a specific building?
5. How many students are enrolled in a specific course?
6. What are the prerequisites for a specific course?
7. What is the contact information for a specific employee?
8. What is the description of a specific course?
9. What is the total number of employees in a specific department?
10. What are the courses offered by a specific department?
11. What is the office location of a specific employee?
12. What are the courses offered in a specific academic term?
13. What is the total number of courses offered in the current academic year?
14. What is the total number of students enrolled in a specific term?
15. What is the total number of courses

SELECT (COUNT(DISTINCT TIP_SUBJECT_OFFERED.SUBJECT_ID) / (SELECT COUNT(DISTINCT WAREUSER.CIS_COURSE_CATALOG.SUBJECT_ID) FROM WAREUSER.CIS_COURSE_CATALOG WHERE WAREUSER.CIS_COURSE_CATALOG.EFFECTIVE_TERM_CODE = 'given_term_code')) * 100 AS PERCENTAGE FROM TIP_SUBJECT_OFFERED WHERE TIP_SUBJECT_OFFERED.TERM_CODE = 'given_term_code'
SELECT (COUNT(DISTINCT WAREUSER.TIP_SUBJECT_OFFERED.SUBJECT_ID) / (SELECT COUNT(DISTINCT WAREUSER.CIS_COURSE_CATALOG.SUBJECT_ID) FROM WAREUSER.CIS_COURSE_CATALOG WHERE WAREUSER.CIS_COURSE_CATALOG.EFFECTIVE_TERM_CODE = 'given_term_code')) * 100 AS PERCENTAGE FROM WAREUSER.TIP_SUBJECT_OFFERED WHERE WAREUSER.TIP_SUBJECT_OFFERED.TERM_CODE = 'given_term_code'
An error occurred: ORA-00937: not a single-group group function
An error occurred: ORA-00937: not a single-group group function
['First error without replacing tables: ORA-00937: not a single-group group function \n Second Error when replacing tables: ORA-00937: not a single-group group function.']
['Not support

In [15]:
df_q['responses_gpt_global_schema'] = gold_answers
df_q['failures_gpt_global_schema'] = failures

## Continue with the answers from gpt with schema information and query logs

In [ ]:
gold_answers = []
failures = []
for qs in df_q['gpt_sql_global_schema_and_logs']:
    gold_answer = []
    failure = []
    for q in qs:
        try:
            gold_answer.append(execute_query(q))
            failure.append(None)
        except Excpetion as e:
            gold_answer.append(None)
            failure.append(str(e))
    gold_answers.append(gold_answer)
    failures.append(failure)

In [ ]:
df_q['responses_gpt_global_schema_and_logs'] = gold_answers
df_q['failures_gpt_global_schema_and_logs'] = failures

# Calculate Accuracy, Precision, Recall and F1-Score

Accuracy: $\frac{\text{Number of correct predictions}}{\text{Total number of predictions}}$

Precision: $\frac{\text{True Positives}}{\text{True Positives} + \text{False Positives}}$

Recall: $\frac{\text{True Positives}}{\text{True Positives} + \text{False Negatives}}$

F1-Score: $2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}$

In [23]:
def metric_(gold_set, prediction_set, recall=None):
  if gold_set is None or prediction_set is None:
    return np.array([0, 0, 0])
  if len(gold_set) == len(prediction_set) == 0:
    return np.array([1, 1, 1])

  true_positives = len(gold_set.intersection(prediction_set))
  false_positives = len(prediction_set - gold_set)
  false_negatives = len(gold_set - prediction_set)

  assert true_positives + false_positives == len(prediction_set)
  assert true_positives + false_negatives == len(gold_set)

  precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) != 0 else 1

  if recall is None:
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) != 0 else 0
  f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
  return np.array([precision, recall, f1])

In [29]:
def calculate_metrics_for(gpt_format:str):
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    if 'expected_responses' in df_q.keys():
        for i in range(len(df_q)):
            accuracy = []
            precision = []
            recall = []
            f1_score = []
            for j in range(len(df_q[gpt_format][i])):
                accuracy_ = []
                precision_ = []
                recall_ = []
                f1_score_ = []
                for h in range(len(df_q['sql'][i])):
                    if df_q['expected_responses'][i][h] is None or df_q[gpt_format][i][j] is None:
                        accuracy_val, precision_val, recall_val, f1_val =  0,0,0,0
                    else:
                        gold = set(df_q['expected_responses'][i][h])
                        prediction = set(df_q[gpt_format][i][j])
                        accuracy_val = int(gold == prediction)
                        precision_val, recall_val, f1_val = metric_(gold, prediction)
                    accuracy_.append(accuracy_val)
                    precision_.append(precision_val)
                    recall_.append(recall_val)
                    f1_score_.append(f1_val)
                    if accuracy_val == 1:
                        break
                if accuracy_val == 1:
                    accuracy.append(accuracy_val)
                    precision.append(precision_val)
                    recall.append(recall_val)
                    f1_score.append(f1_val)
                else:
                    accuracy.append(max(accuracy_))
                    precision.append(max(precision_))
                    recall.append(max(recall_))
                    f1_score.append(max(f1_score_))
            accuracies.append(accuracy)
            precisions.append(precision)
            recalls.append(recall)
            f1_scores.append(f1_score)
    return [accuracies, precisions, recalls, f1_scores]


In [30]:
accuracies_nocontext, precisions_nocontext, recalls_nocontext, f1_scores_nocontext = [], [], [], []
accuracies_nocontext, precisions_nocontext, recalls_nocontext, f1_scores_nocontext = calculate_metrics_for('responses_gpt_nocontent')
df_q['accuracies_nocontext'] = accuracies_nocontext
df_q['precisions_nocontext'] = precisions_nocontext
df_q['recalls_nocontext'] = recalls_nocontext
df_q['f1_scores_nocontext'] = f1_scores_nocontext

In [31]:
accuracies_global_schema, precisions_global_schema, recalls_global_schema, f1_scores_global_schema = calculate_metrics_for('responses_gpt_global_schema')
df_q['accuracies_global_schema'] = accuracies_global_schema
df_q['precisions_global_schema'] = precisions_global_schema
df_q['recalls_global_schema'] = recalls_global_schema
df_q['f1_scores_global_schema'] = f1_scores_global_schema

## old accuracies

In [225]:
arrray_of_metrics = ['accuracies', 'precisions', 'recalls', 'f1_scores']
for metric in arrray_of_metrics:
    print(metric + ' for nocontext: ' + str(sum([sum(a) for a in df_q[metric + '_nocontext']])/(len(df_q))))

accuracies for nocontect: 0.0
precisions for nocontect: 0.0
recalls for nocontect: 0.0
f1_scores for nocontect: 0.0


In [227]:
for metric in arrray_of_metrics:
    print(metric + ' for global schema: ' + str(sum([sum(a) for a in df_q[metric + '_global_schema']])/(len(df_q))))

accuracies for global schema: 0.26
precisions for global schema: 0.5003921568627451
recalls for global schema: 0.29496752758956857
f1_scores for global schema: 0.3009328845686428


In [27]:
arrray_of_metrics = ['accuracies', 'precisions', 'recalls', 'f1_scores']
for metric in arrray_of_metrics:
    print(metric + ' for nocontext: ' + str(sum([sum(a) for a in df_q[metric + '_nocontext']])/(len(df_q))))

accuracies for nocontext: 0.0
precisions for nocontext: 0.0
recalls for nocontext: 0.0
f1_scores for nocontext: 0.0


In [28]:
for metric in arrray_of_metrics:
    print(metric + ' for global schema: ' + str(sum([sum(a) for a in df_q[metric + '_global_schema']])/(len(df_q))))

accuracies for global schema: 0.0
precisions for global schema: 0.16666666666666666
recalls for global schema: 0.0
f1_scores for global schema: 0.0


# Save Dataframe with generated answers


In [ ]:
df_q.to_csv(str(prefix) + '/data/gpt_eval_Brandin_v0.1.csv', index=False)

## Save information in file

In [2]:
def save_accuracies(variant:str):
    current_datetime = datetime.datetime.now()
    file_content = '-->' + variant + '-' + str(current_datetime)+ ':'
    arrray_of_metrics = ['accuracies', 'precisions', 'recalls', 'f1_scores']
    for metric in arrray_of_metrics:
        file_content += metric + ' for nocontext: ' + str(sum([sum(a) for a in df_q[metric + '_nocontext']])/(len(df_q))) + '\n'
    for metric in arrray_of_metrics:
        file_content += metric + ' for global schema: ' + str(sum([sum(a) for a in df_q[metric + '_global_schema']])/(len(df_q)))+ '\n'
    with open(str(prefix) + '/data/evaluation.txt', "a") as file:
        file.write(file_content)

'2024-04-26 17:05:15.566718'

In [ ]:
sav_accuracies()